<img src="https://github.com/nicholasmetherall/digital-earth-pacific-macblue-activities/blob/main/attachments/images/DE_Pacific_banner.JPG?raw=true" width="900"/>

Figure 1.1.a. Jupyter environment + Python notebooks

# Digital Earth Pacific Notebook 1 prepare postcard and load data to csv

The objective of this notebook is to prepare a geomad postcard for your AOI (masking, scaling and loading additional band ratios and spectral indices) and sampling all the datasets into a csv based on your training data geodataframe.

## Step 1.1: Configure the environment

In [21]:
from datetime import datetime
import geopandas as gpd
import numpy as np
import pandas as pd
import joblib
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

In [22]:
# Predefined variable for title and version

# Enter your initials
initials = "nm"

# site
site = "tongatapu"

# Date
date = datetime.now()

# Make a clean version string
version = f"{initials}-{site}-{date.strftime('%d%m%Y')}"
print(version)

nm-tongatapu-09122025


### Postcard csv

The objective of this notebook was to train the machine learning model that will allow us to classify an area with land cover classes defined through the training data.

Step 1.2. Input the training data to sample geomad data from the postcard

In [23]:
joined_df = gpd.read_file(f"training-data/{version}-training.csv")
joined_df = joined_df.astype("float32")
joined_df

,LULC_code,nir,red,blue,green,emad,smad,bcmad,nir08,nir09,...,cai,ndvi,evi,savi,ndwi,mci,ndci,nbi,y,x
0,5.0,0.3070,0.1063,0.0852,0.1147,0.079907,1.329660e-07,0.000005,0.3332,0.3604,...,-0.085350,0.485604,0.506356,0.485604,-0.456011,1.996099,0.182622,-0.125986,-2393675.0,3876145.0
1,3.0,0.3419,0.0842,0.0633,0.0989,0.156016,6.550729e-07,0.000010,0.3326,0.3506,...,-0.046281,0.604788,0.561482,0.604788,-0.551270,2.274784,0.281876,-0.234519,-2404505.0,3875165.0
2,3.0,0.2105,0.0710,0.0717,0.0841,0.170746,1.638007e-06,0.000012,0.2695,0.3155,...,0.156966,0.495560,0.414334,0.495560,-0.429056,1.585090,0.303238,-0.053027,-2404585.0,3875195.0
3,3.0,0.3322,0.1085,0.0794,0.1169,0.133435,6.234050e-07,0.000008,0.3098,0.1713,...,0.057007,0.507602,0.520285,0.507602,-0.479403,2.038037,0.200737,-0.115327,-2404405.0,3875085.0
4,3.0,0.2817,0.1015,0.0749,0.1016,0.130518,6.958604e-07,0.000008,0.2729,0.2862,...,0.056081,0.470251,0.453554,0.470251,-0.469867,1.978230,0.167692,-0.062017,-2404345.0,3875065.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
570,3.0,0.4121,0.0907,0.0691,0.1135,0.173982,4.852354e-07,0.000008,0.4341,0.4378,...,-0.019174,0.639220,0.653350,0.639220,-0.568113,2.270523,0.333578,-0.092090,-2405355.0,3880255.0
571,3.0,0.4047,0.0822,0.0641,0.1079,0.148792,4.859924e-07,0.000007,0.4300,0.4168,...,0.077034,0.662354,0.655929,0.662354,-0.579009,2.355646,0.352756,-0.105889,-2405235.0,3880205.0
572,1.0,0.4006,0.0349,0.0336,0.0579,0.132552,1.591623e-07,0.000008,0.4103,0.4384,...,0.004444,0.839724,0.711782,0.839724,-0.747437,4.138430,0.470008,-0.360734,-2405735.0,3880015.0
573,1.0,0.3680,0.0374,0.0353,0.0554,0.110063,1.613677e-07,0.000007,0.4022,0.4148,...,-0.020231,0.815491,0.664784,0.815491,-0.738309,3.845350,0.438017,-0.316637,-2406105.0,3880525.0


In [24]:
joined_df=joined_df.drop(columns=["y", "x"])

In [25]:
print(len(joined_df.columns))
joined_df.columns

26


Index(['LULC_code', 'nir', 'red', 'blue', 'green', 'emad', 'smad', 'bcmad',
       'nir08', 'nir09', 'swir16', 'swir22', 'coastal', 'rededge1', 'rededge2',
       'rededge3', 'mndwi', 'ndti', 'cai', 'ndvi', 'evi', 'savi', 'ndwi',
       'mci', 'ndci', 'nbi'],
      dtype='object')

In [26]:
# The classes are the first column
classes = np.array(joined_df)[:, 0]

# The observation data is everything after the second column
observations = np.array(joined_df)[:, 1:]

# Create a model...
classifier = RandomForestClassifier()

# ...and fit it to the data
model = classifier.fit(observations, classes)

In [27]:
# Dynamically create the filename with f-string
file_path = f"models/{version}-test.model"

# Save the model
joblib.dump(model, file_path)

['models/nm-tongatapu-09122025-test.model']